In [38]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime 
import pyodbc
import sqlalchemy as sal


In [2]:
url = 'https://www.psx.com.pk/market-summary/#main'

In [3]:
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
container = soup.find('div', class_='col-sm-12 tab-pane inner-content-table automobile-div active')


In [6]:
rows = container.find_all('tr', class_='red-text-td')

In [7]:
dictionary = {
    'titles':[],
    'ldcp': [],
    'opens': [],
    'high': [],
    'low': [],
    'current': [],
    'change': [],
    'volume': [],
    'scrap_time':[]
}

for row in rows:
    a = row.find_all('td')
    dictionary['titles'].append(a[0].text.strip())
    dictionary['ldcp'].append(a[1].text.strip())
    dictionary['opens'].append(a[2].text.strip())
    dictionary['high'].append(a[3].text.strip())
    dictionary['low'].append(a[4].text.strip())
    dictionary['current'].append(a[5].text.strip())
    dictionary['change'].append(a[6].text.strip())
    dictionary['volume'].append(a[7].text.strip())
    dictionary['scrap_time'].append(datetime.datetime.now())

In [43]:
def extract_data():
    url = 'https://www.psx.com.pk/market-summary/#main'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    container = soup.find('div', class_='col-sm-12 tab-pane inner-content-table automobile-div active')
    rows = container.find_all('tr', class_='red-text-td')
    dictionary = {
    'titles':[],
    'ldcp': [],
    'opens': [],
    'high': [],
    'low': [],
    'current': [],
    'change': [],
    'volume': [],
    'scrap_time':[]
    }

    for row in rows:
        a = row.find_all('td')
        dictionary['titles'].append(a[0].text.strip())
        dictionary['ldcp'].append(a[1].text.strip())
        dictionary['opens'].append(a[2].text.strip())
        dictionary['high'].append(a[3].text.strip())
        dictionary['low'].append(a[4].text.strip())
        dictionary['current'].append(a[5].text.strip())
        dictionary['change'].append(a[6].text.strip())
        dictionary['volume'].append(a[7].text.strip())
        dictionary['scrap_time'].append(datetime.datetime.now())
    print(f'✅ Data of {len(dictionary['titles'])} lines extracted successfully.')
    return dictionary

In [48]:
def transform_data(dictionary):
    df = pd.DataFrame(dictionary)
    df = df.replace(',', '', regex=True)
    df = df.replace('', pd.NA)
    type_conv = {
    'ldcp': float,
    'opens': float,
    'high': float,
    'low': float,
    'current': float,
    'change': float,
    'volume': float
    }

    for i in type_conv:
        df[i] = pd.to_numeric(df[i], errors='coerce')
    df['day_range'] = df['high'] - df['low']
    df['volatility_perc'] = ((df['high'] - df['low'])/ df['opens'])*100
    df_volatile = df.nlargest(5, 'volatility_perc')
    df_high_performance = df.nlargest(5, 'change')
    df_low_performance = df.nsmallest(5, 'change')
    return df, df_volatile, df_high_performance, df_low_performance

In [8]:
df = pd.DataFrame(dictionary)
df

,titles,ldcp,opens,high,low,current,change,volume,scrap_time
0,Dewan Auto Engg,31.62,29.4,33.76,29.4,31.2,-0.5,"22,328",2025-05-03 22:07:57.656480
1,Siemens Pak.,"1,500.00",1510.0,1510.0,1510.0,1510.0,,1,2025-05-03 22:07:57.656480
2,Power Cem(Pref),15.23,16.75,16.75,15.16,15.16,-0.07,"4,100",2025-05-03 22:07:57.656480
3,Berger Paints,94.99,96.0,97.84,93.01,94.8,-0.94,"82,947",2025-05-03 22:07:57.656480
4,Engro Poly (Pref),11.99,11.5,11.95,11.5,11.95,-0.04,"7,000",2025-05-03 22:07:57.656480
...,...,...,...,...,...,...,...,...,...
114,Sunrays Textile,170.30,153.27,174.9,153.27,157.8,-8.18,"185,692",2025-05-03 22:07:57.669229
115,Prosperity Weaving,35.00,35.0,35.01,34.0,34.0,,153,2025-05-03 22:07:57.669229
116,Shahtaj Textile,69.88,75.85,75.85,62.89,72.05,,257,2025-05-03 22:07:57.669229
117,Zephyr Textile,12.49,11.24,11.99,11.24,11.99,-1.12,600,2025-05-03 22:07:57.669229


In [9]:
df.dtypes

titles                object
ldcp                  object
opens                 object
high                  object
low                   object
current               object
change                object
volume                object
scrap_time    datetime64[ns]
dtype: object

In [10]:
df = df.replace(',', '', regex=True)
df = df.replace('', pd.NA)

In [11]:
type_conv = {
    'ldcp': float,
    'opens': float,
    'high': float,
    'low': float,
    'current': float,
    'change': float,
    'volume': float
}

for i in type_conv:
    df[i] = pd.to_numeric(df[i], errors='coerce')

In [12]:
df.dtypes

titles                object
ldcp                 float64
opens                float64
high                 float64
low                  float64
current              float64
change               float64
volume                 int64
scrap_time    datetime64[ns]
dtype: object

In [13]:
df['change'].nlargest(5)

49   -0.01
28   -0.02
48   -0.02
61   -0.02
63   -0.02
Name: change, dtype: float64

In [14]:
df['opens'].nlargest(5)

17     23500.00
1       1510.00
43      1500.02
118     1193.00
91      1073.50
Name: opens, dtype: float64

In [15]:
df['day_range'] = df['high'] - df['low']

In [16]:
df

,titles,ldcp,opens,high,low,current,change,volume,scrap_time,day_range
0,Dewan Auto Engg,31.62,29.40,33.76,29.40,31.20,-0.50,22328,2025-05-03 22:07:57.656480,4.36
1,Siemens Pak.,1500.00,1510.00,1510.00,1510.00,1510.00,NaN,1,2025-05-03 22:07:57.656480,0.00
2,Power Cem(Pref),15.23,16.75,16.75,15.16,15.16,-0.07,4100,2025-05-03 22:07:57.656480,1.59
3,Berger Paints,94.99,96.00,97.84,93.01,94.80,-0.94,82947,2025-05-03 22:07:57.656480,4.83
4,Engro Poly (Pref),11.99,11.50,11.95,11.50,11.95,-0.04,7000,2025-05-03 22:07:57.656480,0.45
...,...,...,...,...,...,...,...,...,...,...
114,Sunrays Textile,170.30,153.27,174.90,153.27,157.80,-8.18,185692,2025-05-03 22:07:57.669229,21.63
115,Prosperity Weaving,35.00,35.00,35.01,34.00,34.00,NaN,153,2025-05-03 22:07:57.669229,1.01
116,Shahtaj Textile,69.88,75.85,75.85,62.89,72.05,NaN,257,2025-05-03 22:07:57.669229,12.96
117,Zephyr Textile,12.49,11.24,11.99,11.24,11.99,-1.12,600,2025-05-03 22:07:57.669229,0.75


In [17]:
df['volatility_perc'] = ((df['high'] - df['low'])/ df['opens'])*100

In [18]:
df

,titles,ldcp,opens,high,low,current,change,volume,scrap_time,day_range,volatility_perc
0,Dewan Auto Engg,31.62,29.40,33.76,29.40,31.20,-0.50,22328,2025-05-03 22:07:57.656480,4.36,14.829932
1,Siemens Pak.,1500.00,1510.00,1510.00,1510.00,1510.00,NaN,1,2025-05-03 22:07:57.656480,0.00,0.000000
2,Power Cem(Pref),15.23,16.75,16.75,15.16,15.16,-0.07,4100,2025-05-03 22:07:57.656480,1.59,9.492537
3,Berger Paints,94.99,96.00,97.84,93.01,94.80,-0.94,82947,2025-05-03 22:07:57.656480,4.83,5.031250
4,Engro Poly (Pref),11.99,11.50,11.95,11.50,11.95,-0.04,7000,2025-05-03 22:07:57.656480,0.45,3.913043
...,...,...,...,...,...,...,...,...,...,...,...
114,Sunrays Textile,170.30,153.27,174.90,153.27,157.80,-8.18,185692,2025-05-03 22:07:57.669229,21.63,14.112351
115,Prosperity Weaving,35.00,35.00,35.01,34.00,34.00,NaN,153,2025-05-03 22:07:57.669229,1.01,2.885714
116,Shahtaj Textile,69.88,75.85,75.85,62.89,72.05,NaN,257,2025-05-03 22:07:57.669229,12.96,17.086355
117,Zephyr Textile,12.49,11.24,11.99,11.24,11.99,-1.12,600,2025-05-03 22:07:57.669229,0.75,6.672598


In [19]:
df['volatility_perc'].nlargest(5)

77     27.034884
37     20.979021
55     19.907763
108    19.638243
35     18.995516
Name: volatility_perc, dtype: float64

In [20]:
df_volatile = df.nlargest(5, 'volatility_perc')

In [21]:
df_high_performance = df.nlargest(5, 'change')

In [22]:
df_low_performance = df.nsmallest(5, 'change')

def create_connection():
    conn_str = 'mssql://DESKTOP-QMACISC/airflowProject?driver=ODBC+DRIVER+17+FOR+SQL+SERVER'

    try:
        engine = sal.create_engine(conn_str)
        conn = engine.connect()
        print("✅ Connected to SQL Server successfully!")
        return conn
    except Exception as e:
        print(f"❌ Error connecting to SQL Server: {e}")
        return None

def insert_data():
    conn = create_connection()
    if not conn:
        return
    try:
        df.to_sql(f'psx_info', con=conn, index=True, index_label='id', if_exists='append')
        print(f"✅ Inserted {len(df)} rows into database.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")

    
    try:
        df_volatile.to_sql(f'psx_volatile', con=conn, index=True, index_label='id',  if_exists='append')
        print(f"✅ Inserted {len(df_volatile)} rows into database.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")

    
    try:
        df_high_performance.to_sql(f'psx_high_performance', con=conn, index=True, index_label='id', if_exists='append')
        print(f"✅ Inserted {len(df_high_performance)} rows into database.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")

    
    try:
        df_low_performance.to_sql(f'psx_low_performance', con=conn, index=True, index_label='id', if_exists='append')
        print(f"✅ Inserted {len(df_low_performance)} rows into database.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")    
    

✅ Connected to SQL Server successfully!
✅ Inserted 119 rows into database.
✅ Inserted 5 rows into database.
✅ Inserted 5 rows into database.
✅ Inserted 5 rows into database.


In [44]:
extract_data()


✅ Data of 119 lines extracted successfully.


{'titles': ['Dewan Auto Engg',
  'Siemens Pak.',
  'Power Cem(Pref)',
  'Berger Paints',
  'Engro Poly (Pref)',
  'Ittehad Chemicals',
  'Sitara Peroxide',
  'Bank Makramah',
  'Samba Bank',
  'St.Chart.Bank',
  'Aisha StelCoP/S',
  'Dadex Eternit',
  'Metro Steel',
  'Mughal Iron',
  'Alfalah Consumer',
  'National Foods',
  'Shield Corp.',
  'Unilever FoodsXD',
  'ZIL Limited',
  'AGP-MAYB',
  'CNERGY-MAY',
  'GATM-MAY',
  'OCTOPUS-MAY',
  'SNGP-MAY',
  'SYS-MAYB',
  'Ask.Gen.Insur.XD',
  'Askari Life Ass',
  'EFU Life AssuranceXD',
  'Habib Ins.XD',
  'Pak Gen.Ins.',
  'PICIC Ins.Ltd.',
  '786 Invest Ltd',
  'Apna Microfin.',
  'Calcorp Limited',
  'Dawood Equities',
  'Dawood LawXD',
  'First Cap.Equit',
  'Imperial Limite',
  'JahangirSidd(Pref)XD',
  'JS Global Cap.',
  'LSE Fin. Services',
  'Sec. Inv. BankXD',
  'Trust Brokerage',
  'Bata (Pak) Ltd.',
  'Diamond Ind.',
  'GOC (Pak) Ltd.',
  'Olympia Mills',
  'Pak Services',
  'Tri-Pack Films',
  'UDL Int.Ltd.',
  'United Brand

In [49]:
transform_data(dictionary)

(                 titles     ldcp    opens     high      low  current  change  \
 0       Dewan Auto Engg    31.62    29.40    33.76    29.40    31.20   -0.50   
 1          Siemens Pak.  1500.00  1510.00  1510.00  1510.00  1510.00     NaN   
 2       Power Cem(Pref)    15.23    16.75    16.75    15.16    15.16   -0.07   
 3         Berger Paints    94.99    96.00    97.84    93.01    94.80   -0.94   
 4     Engro Poly (Pref)    11.99    11.50    11.95    11.50    11.95   -0.04   
 ..                  ...      ...      ...      ...      ...      ...     ...   
 114     Sunrays Textile   170.30   153.27   174.90   153.27   157.80   -8.18   
 115  Prosperity Weaving    35.00    35.00    35.01    34.00    34.00     NaN   
 116     Shahtaj Textile    69.88    75.85    75.85    62.89    72.05     NaN   
 117      Zephyr Textile    12.49    11.24    11.99    11.24    11.99   -1.12   
 118       Pak TobaccoXD  1192.39  1193.00  1199.98  1145.25  1160.00  -30.05   
 
      volume              

In [50]:
insert_data()

✅ Connected to SQL Server successfully!
✅ Inserted 119 rows into database.
✅ Inserted 5 rows into database.
✅ Inserted 5 rows into database.
✅ Inserted 5 rows into database.
